<a href="https://colab.research.google.com/github/643020514-7/python_dateviz2023/blob/main/DATA_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import pandas as pd
import os

In [19]:
from google.colab import drive #conecting google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
os.chdir('/content/drive/MyDrive/Dataviz2023') #path to folder
!pwd  #chacking path

/content/drive/MyDrive/Dataviz2023


In [ ]:
!ls #chacking file in folder from path

#Open exel file

In [32]:
file_name = 'ectreport66.xlsx'
dfs = pd.ExcelFile(file_name)
dfs.sheet_names

['Schema',
 'result_constituencies_PartyList',
 'result_constituencies_Candidate',
 'result_constituencies_status',
 'info_province',
 'info_party_overview',
 'info_constituency',
 'Candidate_Constituency',
 'Candidate_PartyList',
 'Candidate_PM']

In [35]:
Schema = dfs.parse('Schema')
result_constituencies_PartyList = dfs.parse('result_constituencies_PartyList')
result_constituencies_Candidate = dfs.parse('result_constituencies_Candidate')
result_constituencies_status = dfs.parse('result_constituencies_status')
info_province = dfs.parse('info_province')
info_party_overview = dfs.parse('info_party_overview')
info_constituency = dfs.parse('info_constituency')
Candidate_constituency = dfs.parse('Candidate_Constituency')
Candidate_partyList = dfs.parse('Candidate_PartyList')
Candidate_pm = dfs.parse('Candidate_PM')

In [ ]:
Schema

#Explore data

In [ ]:
Schema.head()

In [ ]:
print(result_constituencies_PartyList.isnull().any())

In [ ]:
result_constituencies_Candidate.head()

#info_party_overview

In [ ]:
print(info_party_overview.isnull().any())
info_party_overview.head()

In [47]:
info_party_overview[info_party_overview['party_no'].isnull()] #show row is NaN

,id,party_no,name,abbr,color,logo_url


In [46]:
info_party_overview['party_no']= info_party_overview['party_no'].fillna(99) #repleace 99 to NaN

#info_constituency

In [45]:
info_constituency[info_constituency['zone'].isnull()] #แสดง row ที่มี party_no ว่าง
# zone คือ อาณาเขตการปกครองของเขตเลือกตั้ง
# RNG - ระนอง // มีเขตเดียว อำเภอเมืองระนอง
# SBR - สิงห์บุรี // อำเภอเมืองสิงห์บุรี
# SKM - สมุทรสงคราม // อำเภอเมืองสมุทรสงคราม
# TRT - ตราด // อำเภอเมืองตราด

,cons_id,cons_no,prov_id,registered_vote,total_vote_stations,zone
804,RNG_1,1,RNG,140024,209,NaN
817,SBR_1,1,SBR,168055,213,NaN
839,SKM_1,1,SKM,156558,257,NaN
1037,TRT_1,1,TRT,175112,314,NaN


In [51]:
import numpy as np
# แทนค่าว่าง
info_constituency['zone'] = np.where(info_constituency['prov_id'] == 'RNG', 'อำเภอเมืองระนอง', info_constituency['zone'])
info_constituency['zone'] = np.where(info_constituency['prov_id'] == 'SBR', 'อำเภอเมืองสิงห์บุรี', info_constituency['zone'])
info_constituency['zone'] = np.where(info_constituency['prov_id'] == 'SKM', 'อำเภอเมืองสมุทรสงคราม', info_constituency['zone'])
info_constituency['zone'] = np.where(info_constituency['prov_id'] == 'TRT', 'อำเภอเมืองตราด', info_constituency['zone'])

# 🔗 simple query database
### ข้อมูลที่เราต้องการ
* จำนวนผู้มีสิทธิเลือกตั้ง
* จำนวนผู้มาใช้สิทธิ์ทั้งหมด
* รวมคะแนนเลือกตั้งทั้งหมด
* ร้อยละของผู้ที่มาใช้สิทธิเลือกตั้ง
* จำนวนหน่วยเลือกตั้งทั้งหมด

In [52]:
# จำนวนผู้มีสิทธิเลือกตั้ง
info_constituency['registered_vote'].sum()

148739554

In [53]:
#จำนวนผู้มาใช้สิทธิ์ทั้งหมด
result_constituencies_status['turn_out'].sum()

39284752

In [54]:
#รวมคะแนนเลือกตั้งทั้งหมด
info_constituency['total_vote_stations'].sum()

282776

In [55]:
#ร้อยละของผู้ที่มาใช้สิทธิเลือกตั้ง
result_constituencies_status['percent_turn_out'].mean()

75.165671125

In [56]:
#จำนวนหน่วยเลือกตั้งทั้งหมด
result_constituencies_Candidate['mp_app_vote'].sum()

36595160

# 1-2 midterm examination

# 🔗 รวมตารางเพื่อนำไปทำ Dashbroad
* จำนวนโหวตที่จังหวัดนั้นโหวตให้พรรคการเมือง
* ผลตามจังหวัดแบบภาพรวม // Top-5
* จำนวนผู้มาใช้สิทธิ์
* ร้อยละคะแนนเสียง ส.ส. เขตของพรรคนั้นต่อจำนวนผู้ใช้สิทธิ์ทั้งหมด
* ผลแบบภาพรวม ตามพรรคที่ได้คะแนนอันดับ // Top-5
* เรียงลำดับจังหวัดที่มีผู้มีสิทธิเลือกตั้งมากที่สุด
* จำนวนที่ออกมาเลือกตั้ง
* ร้อยละของผู้ออกมาใช้สิทธิ์
* พรรคการเมืองที่จังหวัดนั้นโหวตให้มากที่สุด

In [57]:
# (3) x (6)
table_merge = result_constituencies_Candidate.merge(Candidate_constituency, how ='left', left_on='mp_app_id',right_on='mp_app_id')
print(table_merge.shape)
table_merge.head()

(4781, 10)


,cons_id,mp_app_id,mp_app_rank,mp_app_vote,mp_app_vote_percent,party_id,mp_app_name,mp_app_no,mp_app_party_id,image_url
0,ACR_1,ACR_1_1,9,459,0.43477,706,นางสาวปิยะมาศ บัวแก้ว,1,706,https://cdn.ectreport.com/images/mp_candidates...
1,ACR_1,ACR_1_10,1,38275,36.25419,709,นางสุขสมรวย วันทนียกุล,10,709,https://cdn.ectreport.com/images/mp_candidates...
2,ACR_1,ACR_1_11,13,126,0.11935,712,นายการันต์ ลุนพันธ์,11,712,https://cdn.ectreport.com/images/mp_candidates...
3,ACR_1,ACR_1_12,12,127,0.12029,742,นายไสว ผลาวงศ์,12,742,https://cdn.ectreport.com/images/mp_candidates...
4,ACR_1,ACR_1_13,11,212,0.20081,761,นายสงบ โสภิพันธ์,13,761,https://cdn.ectreport.com/images/mp_candidates...


In [58]:
# table_merge x (5)
table_merge1 = table_merge.merge(info_party_overview, how ='left', left_on='party_id',right_on='id')
print(table_merge1.shape)
table_merge1.head()

(4781, 16)


,cons_id,mp_app_id,mp_app_rank,mp_app_vote,mp_app_vote_percent,party_id,mp_app_name,mp_app_no,mp_app_party_id,image_url,id,party_no,name,abbr,color,logo_url
0,ACR_1,ACR_1_1,9,459,0.43477,706,นางสาวปิยะมาศ บัวแก้ว,1,706,https://cdn.ectreport.com/images/mp_candidates...,706,14.0,ชาติพัฒนากล้า,ชพก.,#233e8a,https://cdn.ectreport.com/images/logos/ชาติพัฒ...
1,ACR_1,ACR_1_10,1,38275,36.25419,709,นางสุขสมรวย วันทนียกุล,10,709,https://cdn.ectreport.com/images/mp_candidates...,709,7.0,ภูมิใจไทย,ภท.,#0c149c,https://cdn.ectreport.com/images/logos/ภูมิใจไ...
2,ACR_1,ACR_1_11,13,126,0.11935,712,นายการันต์ ลุนพันธ์,11,712,https://cdn.ectreport.com/images/mp_candidates...,712,2.0,ประชาธิปไตยใหม่,ปธม.,#eb4138,https://cdn.ectreport.com/images/logos/ประชาธิ...
3,ACR_1,ACR_1_12,12,127,0.12029,742,นายไสว ผลาวงศ์,12,742,https://cdn.ectreport.com/images/mp_candidates...,742,61.0,คลองไทย,คล.ท.,#0eafeb,https://cdn.ectreport.com/images/logos/คลองไทย...
4,ACR_1,ACR_1_13,11,212,0.20081,761,นายสงบ โสภิพันธ์,13,761,https://cdn.ectreport.com/images/mp_candidates...,761,5.0,พลังสังคมใหม่,พ.ส.ม.,#eabcc3,https://cdn.ectreport.com/images/logos/พลังสัง...


In [59]:
# table_merge1 x (4)
table_merge2 = table_merge1.merge(result_constituencies_status, how ='left', left_on='cons_id',right_on='cons_id')
print(table_merge2.shape)
table_merge2.head()

(4781, 22)


,cons_id,mp_app_id,mp_app_rank,mp_app_vote,mp_app_vote_percent,party_id,mp_app_name,mp_app_no,mp_app_party_id,image_url,...,name,abbr,color,logo_url,prov_id,turn_out,percent_turn_out,counted_vote_stations,percent_count,pause_report
0,ACR_1,ACR_1_1,9,459,0.43477,706,นางสาวปิยะมาศ บัวแก้ว,1,706,https://cdn.ectreport.com/images/mp_candidates...,...,ชาติพัฒนากล้า,ชพก.,#233e8a,https://cdn.ectreport.com/images/logos/ชาติพัฒ...,ACR,105574,72.60136,310,100.0,False
1,ACR_1,ACR_1_10,1,38275,36.25419,709,นางสุขสมรวย วันทนียกุล,10,709,https://cdn.ectreport.com/images/mp_candidates...,...,ภูมิใจไทย,ภท.,#0c149c,https://cdn.ectreport.com/images/logos/ภูมิใจไ...,ACR,105574,72.60136,310,100.0,False
2,ACR_1,ACR_1_11,13,126,0.11935,712,นายการันต์ ลุนพันธ์,11,712,https://cdn.ectreport.com/images/mp_candidates...,...,ประชาธิปไตยใหม่,ปธม.,#eb4138,https://cdn.ectreport.com/images/logos/ประชาธิ...,ACR,105574,72.60136,310,100.0,False
3,ACR_1,ACR_1_12,12,127,0.12029,742,นายไสว ผลาวงศ์,12,742,https://cdn.ectreport.com/images/mp_candidates...,...,คลองไทย,คล.ท.,#0eafeb,https://cdn.ectreport.com/images/logos/คลองไทย...,ACR,105574,72.60136,310,100.0,False
4,ACR_1,ACR_1_13,11,212,0.20081,761,นายสงบ โสภิพันธ์,13,761,https://cdn.ectreport.com/images/mp_candidates...,...,พลังสังคมใหม่,พ.ส.ม.,#eabcc3,https://cdn.ectreport.com/images/logos/พลังสัง...,ACR,105574,72.60136,310,100.0,False


In [60]:
# table_merge2 x (7)
table_merge3 = table_merge2.merge(info_province, how ='left', left_on='prov_id',right_on='prov_id')
print(table_merge3.shape)
table_merge3.head()

(4781, 28)


,cons_id,mp_app_id,mp_app_rank,mp_app_vote,mp_app_vote_percent,party_id,mp_app_name,mp_app_no,mp_app_party_id,image_url,...,percent_turn_out,counted_vote_stations,percent_count,pause_report,province_id,province,eng,abbre_thai,total_registered_vote,total_vote_stations
0,ACR_1,ACR_1_1,9,459,0.43477,706,นางสาวปิยะมาศ บัวแก้ว,1,706,https://cdn.ectreport.com/images/mp_candidates...,...,72.60136,310,100.0,False,37,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641
1,ACR_1,ACR_1_10,1,38275,36.25419,709,นางสุขสมรวย วันทนียกุล,10,709,https://cdn.ectreport.com/images/mp_candidates...,...,72.60136,310,100.0,False,37,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641
2,ACR_1,ACR_1_11,13,126,0.11935,712,นายการันต์ ลุนพันธ์,11,712,https://cdn.ectreport.com/images/mp_candidates...,...,72.60136,310,100.0,False,37,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641
3,ACR_1,ACR_1_12,12,127,0.12029,742,นายไสว ผลาวงศ์,12,742,https://cdn.ectreport.com/images/mp_candidates...,...,72.60136,310,100.0,False,37,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641
4,ACR_1,ACR_1_13,11,212,0.20081,761,นายสงบ โสภิพันธ์,13,761,https://cdn.ectreport.com/images/mp_candidates...,...,72.60136,310,100.0,False,37,อำนาจเจริญ,AMNAT CHAROEN,อจ,301354,641


In [67]:
## เขตการเลือกตั้ง
cons_id = table_merge3['cons_id'].tolist()
cons_no = []
for i in range(len(cons_id)):
    re = cons_id[i].split('_')[-1]
    cons_no.append(re)
table_merge3['cons_no'] = cons_no

In [68]:
table_merge3.to_csv('table_merge_result.csv', index=False)